In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from itertools import chain


In [15]:
from sklearn.metrics import classification_report, accuracy_score
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import torch.nn.utils as utils

In [16]:
torch.backends.cudnn.deterministic = True
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)


In [17]:
import os
import pickle
import torch
from sklearn.metrics import classification_report

In [18]:
from itertools import chain

import torch
import torch.nn as nn
import torch.nn.functional as F


In [19]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [20]:
class RNNEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(RNNEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.rnn_encoder = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)

    def forward(self, x: torch.Tensor, h_0: torch.Tensor)->torch.Tensor:
        # x.shape: Batch x sequence length x input_size
        # h_0.shape:
        # output.shape: batch x sequence length x input_size
        output, h_n = self.rnn_encoder(x, h_0)

        # output.shape: batch x input_size
        output = output.mean(dim=1)
        return output


class CNNEncoder(nn.Module):
    def __init__(self, out_channels: int, kernel_size: tuple):
        super(CNNEncoder, self).__init__()
        self.cnn_encoder = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=kernel_size)

    def forward(self, x: torch.Tensor):
        # x.shape: batch x sequence length x kernel_size[1](input_size)
        # x.shape: batch x 1 x sequence length x kernel_size[1]
        x = x.unsqueeze(dim=1)
        # output.shape: batch x out_channels x sequence length - kernel_size[0] + 1
        output = F.relu(self.cnn_encoder(x))
        # output.shape: batch x out_channels
        output = output.mean(dim=2)
        return output


class DetectModel(nn.Module):
    def __init__(self, input_size,
                 hidden_size, rnn_layers,
                 out_channels, height, cnn_layers,
                 linear_hidden_size, linear_layers, output_size):
        super(DetectModel, self).__init__()
        self.rnn_encoder = RNNEncoder(input_size=input_size, hidden_size=hidden_size, num_layers=rnn_layers)
        self.cnn_encoder = CNNEncoder(out_channels=out_channels, kernel_size=(height, input_size))

        self.linear = nn.Sequential(
            nn.Linear(hidden_size + out_channels, linear_hidden_size), nn.ReLU(inplace=True),
            *chain(*[(nn.Linear(linear_hidden_size, linear_hidden_size), nn.ReLU(inplace=True)) for i in range(linear_layers - 2)]),
            nn.Linear(linear_hidden_size, output_size)
        )

    def forward(self, x, h0):
        # h0 for rnn_encoder
        
        rnn_output = self.rnn_encoder(x, h0)
        cnn_output = self.cnn_encoder(x)
        cnn_output = cnn_output.squeeze()
       
        # output.shape: batch x (hidden_size + out_channels)
        # output.shape: batch x output_size
        output = torch.cat([rnn_output, cnn_output], dim=1)
        output = self.linear(output)

        return output

In [21]:
class NeuralNetwork(nn.Module):

    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.best_acc = 0
        self.patience = 0
        self.init_clip_max_norm = 5.0
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    def forward(self):
        raise NotImplementedError

    def fit(self, X_train_source_wid, X_train_source_id, X_train_user_id, X_train_ruid, y_train, y_train_cred, y_train_rucred,
            X_dev_source_wid, X_dev_source_id, X_dev_user_id, X_dev_ruid, y_dev):

        if torch.cuda.is_available():
            self.cuda()

        batch_size = self.config['batch_size']
        self.optimizer = torch.optim.Adam(self.parameters(), lr=self.config['lr'], weight_decay=self.config['reg'], amsgrad=True) #
        # self.optimizer = torch.optim.Adadelta(self.parameters(), weight_decay=self.config['reg'])

        X_train_source_wid = torch.LongTensor(X_train_source_wid)
        X_train_source_id = torch.LongTensor(X_train_source_id)
        X_train_user_id = torch.LongTensor(X_train_user_id)
        X_train_ruid = torch.LongTensor(X_train_ruid)
        y_train = torch.LongTensor(y_train)
        y_train_cred = torch.LongTensor(y_train_cred)
        y_train_rucred = torch.LongTensor(y_train_rucred)
 
        dataset = TensorDataset(X_train_source_wid, X_train_source_id, X_train_user_id, X_train_ruid, y_train, y_train_cred, y_train_rucred)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        # UPDATE

        model1 = DetectModel(config['input_size'],
                      config['hidden_size'], config['rnn_layers'],
                      config['out_channels'], config['height'], config['cnn_layers'],
                      config['linear_hidden_size'], config['linear_layers'], config['output_size'])
        
        
        
        model1 = model1.to(self.device)
        
        self.optimizer1 = torch.optim.Adam(params= model1.parameters(), lr=config['lr'])
        self.criterion = nn.CrossEntropyLoss(ignore_index=3)
        self.initial_hidden_state = torch.zeros(config['rnn_layers'], config['batch_size'], config['hidden_size'], dtype=torch.float, requires_grad=False).to(self.device)

        
        loss_func = nn.CrossEntropyLoss()
        loss_func2 = nn.CrossEntropyLoss(ignore_index=3) # kya h 
        for epoch in range(1, self.config['epochs']+1):
            print("\nEpoch ", epoch, "/", self.config['epochs'])
            self.train()
            avg_loss = 0
            avg_acc = 0
            for i, data in enumerate(dataloader):
                with torch.no_grad():
                    X_source_wid, X_source_id, X_user_id, X_ruid, batch_y, batch_y_cred, batch_y_rucred = (item.cuda(device=self.device) for item in data)

                self.optimizer.zero_grad()
                logit, ulogit, rulogit, r = self.forward(X_source_wid, X_source_id, X_user_id, X_ruid)

                output = model1(r,  self.initial_hidden_state[:, :r.shape[0], :])
                loss3 = self.criterion(output,  batch_y)
                
                
                loss1 = loss_func(logit, batch_y)
                pub_loss = loss_func(ulogit, batch_y_cred)
                uloss = loss_func2(rulogit.view(-1, rulogit.size(-1)), batch_y_rucred.view(-1))
                loss = loss1 + pub_loss + uloss + loss3

                loss.backward()
                self.optimizer.step()

                corrects = (torch.max(logit, 1)[1].view(batch_y.size()).data == batch_y.data).sum()
                accuracy = 100*corrects/len(batch_y)
                print('Batch[{}] - loss: {:.6f}  acc: {:.4f}%({}/{})'.format(i, loss.item(), accuracy, corrects, batch_y.size(0)))

                avg_loss += loss.item()
                avg_acc += accuracy

                if self.init_clip_max_norm is not None:
                    utils.clip_grad_norm_(self.parameters(), max_norm=self.init_clip_max_norm)

            cnt = y_train.size(0) // batch_size + 1
            print("Average loss:{:.6f} average acc:{:.6f}%".format(avg_loss/cnt, avg_acc/cnt))
            if epoch > self.config['epochs']//2 and self.patience > 2: #
                print("Reload the best model...")
                self.load_state_dict(torch.load(self.config['save_path']))
                now_lr = self.adjust_learning_rate(self.optimizer)
                print(now_lr)
                self.patience = 0
            self.evaluate(X_dev_source_wid, X_dev_source_id, X_dev_user_id, X_dev_ruid, y_dev, epoch)


    def adjust_learning_rate(self, optimizer, decay_rate=.5):
        now_lr = 0
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * decay_rate
            now_lr = param_group['lr']
        return now_lr


    def evaluate(self, X_dev_source_wid, X_dev_source_id, X_dev_user_id, X_dev_ruid, y_dev, epoch):
        y_pred = self.predict(X_dev_source_wid, X_dev_source_id, X_dev_user_id, X_dev_ruid)
        acc = accuracy_score(y_dev, y_pred)
        print("Val set acc:", acc)
        print("Best val set acc:", self.best_acc)

        if epoch >= self.config['epochs']//2 and acc > self.best_acc:  #
            self.best_acc = acc
            self.patience = 0
            torch.save(self.state_dict(), self.config['save_path'])
            print(classification_report(y_dev, y_pred, target_names=self.config['target_names'], digits=5))
            print("save model!!!")
        else:
            self.patience += 1


    def predict(self, X_dev_source_wid, X_dev_source_id, X_dev_user_id, X_dev_ruid):
        if torch.cuda.is_available():
            self.cuda()

        self.eval()
        y_pred = []
        X_dev_source_wid = torch.LongTensor(X_dev_source_wid)
        X_dev_source_id = torch.LongTensor(X_dev_source_id)
        X_dev_user_id = torch.LongTensor(X_dev_user_id)
        X_dev_ruid = torch.LongTensor(X_dev_ruid)

        dataset = TensorDataset(X_dev_source_wid, X_dev_source_id, X_dev_user_id, X_dev_ruid)
        dataloader = DataLoader(dataset, batch_size=32)

        for i, data in enumerate(dataloader):
            with torch.no_grad():
                X_source_wid, X_source_id, X_user_id, \
                X_ruid = (item.cuda(device=self.device) for item in data)

            
            logits, _, _, r = self.forward(X_source_wid, X_source_id, X_user_id, X_ruid)
            
            predicted = torch.max(logits, dim=1)[1]
            y_pred += predicted.data.cpu().numpy().tolist()
        return y_pred


In [22]:
class PGAN(NeuralNetwork):

    def __init__(self, config):
        super(PGAN, self).__init__()
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.config = config
        embedding_weights = config['embedding_weights']
        V, D = embedding_weights.shape
        self.n_heads = config['n_heads']

        self.A_us = config['A_us']
        self.A_uu = config['A_uu']
        embeding_size = config['embeding_size']

        self.word_embedding = nn.Embedding(V, D, padding_idx=0, _weight=torch.from_numpy(embedding_weights)) # V embeddings of 300 Size.
        self.user_embedding = nn.Embedding(config['A_us'].shape[0], embeding_size, padding_idx=0) # Number of users embedding of 100 size.
        self.source_embedding = nn.Embedding(config['A_us'].shape[1], embeding_size)
        self.up_embed = nn.Embedding(config['A_us'].shape[0], 100, padding_idx=0)

        self.convs = nn.ModuleList([nn.Conv1d(300, 100, kernel_size=K) for K in config['kernel_sizes']])
        self.max_poolings = nn.ModuleList([nn.MaxPool1d(kernel_size=config['maxlen'] - K + 1) for K in config['kernel_sizes']])

        self.Wcm = [nn.Parameter(torch.FloatTensor(embeding_size, embeding_size)).cuda() for _ in
                    range(self.n_heads)] # 10 Layers of size 100 * 100
        self.Wam = [nn.Parameter(torch.FloatTensor(embeding_size, embeding_size)).cuda() for _ in
                    range(self.n_heads)]
        self.scale = torch.sqrt(torch.FloatTensor([embeding_size])).cuda() #  // self.n_heads

        self.W1 = nn.Parameter(torch.FloatTensor(embeding_size * self.n_heads, embeding_size)) # 1000 * 100
        self.W2 = nn.Parameter(torch.FloatTensor(embeding_size * self.n_heads, embeding_size))   # 1000 * 100
        self.linear = nn.Linear(400, 200)  # 400 Input features and gives 200 Output Features.
        # Activation Function
        self.dropout = nn.Dropout(config['dropout'])
        self.relu = nn.ReLU()
        self.elu = nn.ELU()

        self.fc_out = nn.Sequential(
            nn.Linear(300 + 2 * embeding_size, 100),
            nn.ReLU(),
            nn.Dropout(config['dropout']),
            nn.Linear(100, config["num_classes"])
        )
        self.fc_user_out = nn.Sequential(
            nn.Linear(embeding_size, 100),
            nn.ReLU(),
            nn.Dropout(config['dropout']),
            nn.Linear(100, 3)
        )
        self.fc_ruser_out = nn.Sequential(
            nn.Linear(embeding_size, 100),
            nn.ReLU(),
            nn.Dropout(config['dropout']),
            nn.Linear(100, 3)
        )
        print(self)
        self.init_weights()

    def init_weights(self):
        init.xavier_normal_(self.user_embedding.weight)
        init.xavier_normal_(self.source_embedding.weight)
        for i in range(self.n_heads):
            init.xavier_normal_(self.Wcm[i])
            init.xavier_normal_(self.Wam[i])

        init.xavier_normal_(self.W1)
        init.xavier_normal_(self.W2)
        init.xavier_normal_(self.linear.weight)
        for name, param in self.fc_out.named_parameters():
            if name.__contains__("weight"):
                init.xavier_normal_(param)
        for name, param in self.fc_user_out.named_parameters():
            if name.__contains__("weight"):
                init.xavier_normal_(param)
        for name, param in self.fc_ruser_out.named_parameters():
            if name.__contains__("weight"):
                init.xavier_normal_(param)

    def user_multi_head(self, X_user, X_user_id, Wcm):
        M = self.source_embedding.weight
        linear1 = torch.einsum("bd,dd,sd->bs", X_user, Wcm, M) / self.scale # Doubt h 
        linear1 = self.relu(linear1) # (batch, |S|)

        A_us = self.A_us[X_user_id.cpu(), :].todense()
        A_us = torch.FloatTensor(A_us).cuda()   # (batch, |S|)

        alpha = F.softmax(linear1 * A_us, dim=-1)
        alpha = self.dropout(alpha)
        return alpha.matmul(M)

    def retweet_user_multi_head(self, X_ruser, X_ruser_id, Wam):
        M = self.user_embedding.weight
        linear1 = torch.einsum("bnd,dd,md->bnm", X_ruser, Wam, M) / self.scale # m x bsz
        linear1 = self.relu(linear1)

        s1, s2 = X_ruser_id.size()
        idx = X_ruser_id.view(-1).cpu()
        A_uu = self.A_uu[idx, :].todense()
        A_uu = torch.FloatTensor(A_uu).view(s1, s2, -1).cuda()

        alpha = F.softmax(linear1 * A_uu, dim=-1)  # m x bsz
        alpha = self.dropout(alpha)
        return alpha.matmul(M)

    def publisher_encoder(self, X_user, X_user_id):
        m_hat = []
        for i in range(self.n_heads):
            m_hat.append(self.user_multi_head(X_user, X_user_id, self.Wcm[i]))

        m_hat = torch.cat(m_hat, dim=-1).matmul(self.W1)
        m_hat = self.elu(m_hat)
        m_hat = self.dropout(m_hat)

        U_hat = m_hat + X_user  # bsz x d
        return U_hat

    def retweet_user_encoder(self, X_ruser, X_ruser_id):  # 0.854  0.878
        '''
        :param X_ruser:  (bsz, num_users, d)
        :param X_ruser_id: (bsz, num_users)
        :return:
        '''
        m_hat = []
        for i in range(self.n_heads):
            m_hat.append(self.retweet_user_multi_head(X_ruser, X_ruser_id, self.Wam[i]))
        m_hat = torch.cat(m_hat, dim=-1).matmul(self.W2)
        m_hat = self.elu(m_hat)
        m_hat = self.dropout(m_hat)

        a_hat = m_hat + X_ruser  # bsz x 20 x d
        return a_hat

    def source_encoder(self, X_source, r_user_rep, user_rep):  #
        linear1 = torch.einsum("bd,bnd->bn", X_source, r_user_rep) # / self.scale
        alpha = F.softmax(linear1, dim=-1)
        retweet_rep = torch.einsum("bn,bnd->bd", alpha, r_user_rep)

        # beta = 0.5
        source_rep = torch.cat([retweet_rep, user_rep,
                                retweet_rep * user_rep,
                                retweet_rep - user_rep], dim=-1)  # .mm(self.W) #
        source_rep = self.linear(source_rep)
        source_rep = self.dropout(source_rep)
        return source_rep

    def text_representation(self, X_word):
        X_word = X_word.permute(0, 2, 1)
        conv_block = []
        for Conv, max_pooling in zip(self.convs, self.max_poolings):
            act = self.relu(Conv(X_word))
            pool = max_pooling(act).squeeze()
            conv_block.append(pool)

        features = torch.cat(conv_block, dim=1)
        features = self.dropout(features)
        return features

    def forward(self, X_source_wid, X_source_id, X_user_id, X_ruser_id):  # , X_composer_id, X_reviewer_id
        '''
        :param X_source_wid size: (batch_size, max_words)
                X_source_id size: (batch_size, )
                X_user_id  size: (batch_size, )
                X_retweet_id  size: (batch_size, max_retweets)
                X_retweet_uid  size: (batch_size, max_retweets)

        :return:
        '''
        X_word = self.word_embedding(X_source_wid)
        X_user = self.user_embedding(X_user_id)
        X_ruser = self.user_embedding(X_ruser_id)
        X_source = self.source_embedding(X_source_id)
        R = self.up_embed(X_ruser_id)
        X_text = self.text_representation(X_word)

        user_rep = self.publisher_encoder(X_user, X_user_id)
        r_user_rep = self.retweet_user_encoder(X_ruser, X_ruser_id)  #
        source_rep = self.source_encoder(X_source, r_user_rep, user_rep)  #
        r_rep = self.retweet_user_encoder(R, X_ruser_id)
        tweet_rep = torch.cat([X_text, source_rep], dim=-1)

        Xt_logit = self.fc_out(tweet_rep)
        Xu_logit = self.fc_user_out(user_rep)
        Xru_logit = self.fc_ruser_out(r_user_rep)

        return Xt_logit, Xu_logit, Xru_logit, r_rep



In [22]:
def load_dataset(task):
    print("task: ", task)

    A_us, A_uu = pickle.load(open("/content/drive/MyDrive/twitter15/relations.pkl", 'rb'))
    X_train_source_wid, X_train_source_id, X_train_user_id, X_train_ruid, y_train, y_train_cred, y_train_rucred, word_embeddings = pickle.load(open("/content/drive/MyDrive/twitter15/train.pkl", 'rb'))
    X_dev_source_wid, X_dev_source_id, X_dev_user_id, X_dev_ruid, y_dev = pickle.load(open("/content/drive/MyDrive/twitter15/dev.pkl", 'rb'))
    X_test_source_wid, X_test_source_id, X_test_user_id, X_test_ruid, y_test = pickle.load(open("/content/drive/MyDrive/twitter15/test.pkl", 'rb'))
    config['maxlen'] = len(X_train_source_wid[0])
    if task == 'twitter15':
        config['n_heads'] = 10
    elif task == 'twitter16':
        config['n_heads'] = 8
    else:
        config['n_heads'] = 7
        config['batch_size'] = 128
        config['num_classes'] = 2
        config['target_names'] = ['NR', 'FR']
    print(config)

    config['embedding_weights'] = word_embeddings
    config['A_us'] = A_us
    config['A_uu'] = A_uu
    return X_train_source_wid, X_train_source_id, X_train_user_id, X_train_ruid, y_train, y_train_cred, y_train_rucred, \
           X_dev_source_wid, X_dev_source_id, X_dev_user_id, X_dev_ruid, y_dev, \
           X_test_source_wid, X_test_source_id, X_test_user_id, X_test_ruid, y_test


def train_and_test(model, task):
    model_suffix = model.__name__.lower().strip("text")
    config['save_path'] = '/content/drive/MyDrive/twitter15/weights.best.' + task + "." + model_suffix

    X_train_source_wid, X_train_source_id, X_train_user_id, X_train_ruid, y_train, y_train_cred, y_train_rucred, \
    X_dev_source_wid, X_dev_source_id, X_dev_user_id, X_dev_ruid, y_dev, \
    X_test_source_wid, X_test_source_id, X_test_user_id, X_test_ruid, y_test = load_dataset(task)

    nn = model(config)
    nn.fit(X_train_source_wid, X_train_source_id, X_train_user_id, X_train_ruid, y_train, y_train_cred, y_train_rucred,
          X_dev_source_wid, X_dev_source_id, X_dev_user_id, X_dev_ruid, y_dev)  #

    print("================================")
    nn.load_state_dict(torch.load(config['save_path']))
    y_pred = nn.predict(X_test_source_wid, X_test_source_id, X_test_user_id, X_test_ruid)
    print(classification_report(y_test, y_pred, target_names=config['target_names'], digits=3))


config = {
    'lr':1e-3,
    'reg':1e-6,
    'embeding_size': 100,
    'batch_size':16,
    'nb_filters':100,
    'kernel_sizes':[3, 4, 5],
    'dropout':0.5,
    'epochs':18,
    'num_classes':4,
    'target_names':['NR', 'FR', 'TR', 'UR']

}


if __name__ == '__main__':
    task = 'twitter15'
    # task = 'twitter16'
    # task = 'weibo'
    model = PGAN
    train_and_test(model, task)


task:  twitter15
{'lr': 0.001, 'reg': 1e-06, 'embeding_size': 100, 'batch_size': 16, 'nb_filters': 100, 'kernel_sizes': [3, 4, 5], 'dropout': 0.5, 'epochs': 18, 'num_classes': 4, 'target_names': ['NR', 'FR', 'TR', 'UR'], 'save_path': '/content/drive/MyDrive/twitter15/weights.best.twitter15.pgan', 'maxlen': 50, 'n_heads': 10}
PGAN(
  (word_embedding): Embedding(2246, 300, padding_idx=0)
  (user_embedding): Embedding(2213, 100, padding_idx=0)
  (source_embedding): Embedding(1490, 100)
  (convs): ModuleList(
    (0): Conv1d(300, 100, kernel_size=(3,), stride=(1,))
    (1): Conv1d(300, 100, kernel_size=(4,), stride=(1,))
    (2): Conv1d(300, 100, kernel_size=(5,), stride=(1,))
  )
  (max_poolings): ModuleList(
    (0): MaxPool1d(kernel_size=48, stride=48, padding=0, dilation=1, ceil_mode=False)
    (1): MaxPool1d(kernel_size=47, stride=47, padding=0, dilation=1, ceil_mode=False)
    (2): MaxPool1d(kernel_size=46, stride=46, padding=0, dilation=1, ceil_mode=False)
  )
  (linear): Linear(in_

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
